In [49]:
import pandas as pd
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

# read in solar forecast data
path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_2/'
solar_values_read = pd.read_csv(path+'pred_solar_values.csv')
solar_values = round(solar_values_read.yhat,3)
solar_values.reset_index(drop = True, inplace = True)

# read in demaind data, where total demand is a fixed building demand and
# an estimated demand based on a proposed schedule for the building
schedule = pd.read_csv(path+'schedule_demand.csv')
avg_building = pd.read_csv(path+'building_demand.csv')
total_demand = schedule.sched_demand + avg_building.build_demand
print(f"Total Solar Generation: {solar_values.sum()} \nTotal Demand: {total_demand.sum()}")

# create data for batteries, including capity, efficency and initial charges
# also define time periods,
batteries = ["Battery0", "Battery1"]
capacity = {"Battery0": 60, "Battery1": 80} # in Kw
p_loss = {"Battery0": 0.95, "Battery1": 0.9} # proportion
initial = {"Battery0": 0, "Battery1": 0} # in kW
time_periods = range(len(solar_values_read))

Total Solar Generation: 4939.172 
Total Demand: 5250.6


In [50]:
m = gp.Model()
# max charge / discharge 20kw per single period
MAX_FLOW = 20

# DECISION VARIABLES
flow_in = m.addVars(batteries, time_periods, name="flow_in") 
flow_out = m.addVars(batteries, time_periods, name="flow_out")
grid = m.addVars(time_periods, name='grid') # energy acquistata
state = m.addVars(batteries, time_periods, name="state") # quanta energia in ciascuna batteria 
gen = m.addVars(time_periods, name="gen") # qtà di energia solare usata nel tempo t
zwitch = m.addVars(batteries, time_periods, vtype=GRB.BINARY, name='zwitch') # se una batteria sta caricando o scaricando


In [51]:
# CONSTRAINTS
m.addConstrs((gen[t]+grid[t]+gp.quicksum(flow_out[b,t]-flow_in[b,t]*p_loss[b] for b in batteries) == total_demand[t] for t in time_periods), name='power_balance')
m.addConstrs((state[b,0] == flow_in[b,0]*p_loss[b] - flow_out[b,0] for b in batteries), name='initial_state')
m.addConstrs((state[b,t] == state[b,t-1] + flow_in[b,t]*p_loss[b] -  flow_out[b,t] for b in batteries for t in time_periods if t >= 1), name='subsequent_states') # skippo primo time period
m.addConstrs((flow_in['Battery0',t] + flow_in['Battery1',t] + gen[t] <= solar_values[t] for t in time_periods), name = "solar_avail");
# flowin >= 0 or flowout >= 0
# flowin <= z1*C
# flowout <= z2*C
m.addConstrs((flow_in[b,t] <= zwitch[b,t]*MAX_FLOW for b in batteries for t in time_periods), name = "to_charge")
m.addConstrs((flow_out[b,t] <= (1-zwitch[b,t])*MAX_FLOW for b in batteries for t in time_periods), name = "or_not_to_charge")

# max charge per ciascuna battery in ciascun istante di tempo
for b, t in state:
    state[b,t].UB = capacity[b]

In [52]:
# OBJECTIVE 
# read in estimated price of electricity for each time period
avg_price = pd.read_csv(path+'expected_price.csv')
price = avg_price.price

# FIRST OBJECTIVE
total_grid = grid.sum()
# SECOND OBJECTIVE
total_cost = gp.quicksum(avg_price.price[t]*grid[t] for t in time_periods)

In [53]:
# WEIGHTED MULTI OBJECTIVE
m.setObjectiveN(total_cost, index=0, weight = 1, name='cost')
m.setObjectiveN(total_grid, index=1, weight = 10, name='grid')

m.optimize()
results = pd.DataFrame([[round(v,2) for v in [total_cost.getValue(),total_grid.getValue()]]],
                       columns = ['Cost','GridAmount']
                       )
results


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1440 rows, 1800 columns and 4498 nonzeros
Model fingerprint: 0x9cf62986
Variable types: 1440 continuous, 360 integer (360 binary)
Coefficient statistics:
  Matrix range     [9e-01, 2e+01]
  Objective range  [6e-02, 2e+00]
  Bounds range     [1e+00, 8e+01]
  RHS range        [1e-01, 9e+01]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined)...
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 1440 rows, 1800 columns and 4498 nonzeros
Model f

,Cost,GridAmount
0,616.7,1282.2


In [54]:
# limito il numero di volte che una batteria viene scaricata sotto un certo livello
LIMITE_LOW = 10
v = m.addVars(batteries, time_periods, vtype=GRB.BINARY, name='underlevelbatt')
m.addConstrs((state[b,t] <= 0.3*capacity[b] + (1-v[b,t])*capacity[b] for b in batteries for t in time_periods), name='low variable')
#m.addConstrs((gp.quicksum(v[b,t] for t in time_periods) <= LIMITE_LOW for b in batteries), name='limite_low')  
m.update()

In [55]:
m.setObjectiveN(total_cost, index=0, priority=2, reltol = 0.05, name="cost")
m.setObjectiveN(v.sum(), index=1,  priority=0, reltol = 0.2, name= "depth")
m.setObjectiveN(total_grid, index=2, priority=0, name= "grid")

In [56]:
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1800 rows, 2160 columns and 5218 nonzeros
Model fingerprint: 0xae71baa6
Variable types: 1440 continuous, 720 integer (720 binary)
Coefficient statistics:
  Matrix range     [9e-01, 8e+01]
  Objective range  [6e-02, 2e+00]
  Bounds range     [1e+00, 8e+01]
  RHS range        [1e-01, 1e+02]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives (2 combined)...
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve...
---------------------------------------------------------------------------

Presolve removed 75 rows and 65 columns
Presolve time: 0.01s
Presolved: 1725 r

In [57]:
for o in range(m.NumObj):
  # set which objective we will query
  m.params.ObjNumber = o
  # query the o-th objective value
  print(' ',round(m.ObjNVal,2), end='')

  632.14  0.0  1281.68

In [58]:
mm = gp.Model('mulit_scen') #this defines the model that we'll add to as we finish the formulation
flow_in = mm.addVars(batteries, time_periods, name="flow_in")
flow_out = mm.addVars(batteries, time_periods, name="flow_out")
grid = mm.addVars(time_periods, name="grid")
state = mm.addVars(batteries, time_periods, ub = [capacity[b] for b in batteries for t in time_periods], name="state")
gen = mm.addVars(time_periods, name="gen")
zwitch = mm.addVars(batteries, time_periods, vtype=GRB.BINARY, name="zwitch")

# define a linear expression for total energy purchased from the grid
total_grid = grid.sum()
# define a linear expression for total cost
total_cost = gp.quicksum(avg_price.price[t]*grid[t] for t in time_periods)

power_balance = mm.addConstrs((gp.quicksum(flow_out[b,t] - p_loss[b]*flow_in[b,t] for b in batteries) + gen[t] + grid[t] == total_demand[t]
                  for t in time_periods), name="power_balance")
initial_state = mm.addConstrs((state[b,0] == initial[b] + p_loss[b]*flow_in[b,0] - flow_out[b,0] for b in batteries), name="initial_state")
subsequent_states = mm.addConstrs((state[b,t] == state[b,t-1] + p_loss[b]*flow_in[b,t] - flow_out[b,t] for b in batteries for t in time_periods if t >= 1), name="subsequent_states")
solar_avail = mm.addConstrs((flow_in['Battery0',t] + flow_in['Battery1',t] + gen[t] <= solar_values[t] for t in time_periods), name = "solar_avail")
to_charge = mm.addConstrs((flow_in[b,t] <= 20*zwitch[b,t] for b in batteries for t in time_periods), name = "to_charge")
or_not_to_charge = mm.addConstrs((flow_out[b,t] <= 20*(1-zwitch[b,t]) for b in batteries for t in time_periods), name = "or_not_to_charge")

mm.setObjective(total_cost, GRB.MINIMIZE)
mm.update()

In [59]:
mm.NumScenarios=4
mm.Params.ScenarioNumber = 0
mm.ScenNName = 'Base model'

price2 = avg_price.price2
# change obj
mm.Params.ScenarioNumber = 1
mm.ScenNName = 'Increased price'
for t in time_periods:
  grid[t].ScenNObj = price2[t]


capacity2 = {'Battery0': 48, 'Battery1': 64}
# change variable bound
mm.Params.ScenarioNumber = 2
mm.ScenNName = 'Low Battery'
for b in batteries:
  for t in time_periods:
    state[b,t].ScenNUB = capacity2[b]


solar_values2 = round(0.1*solar_values_read.yhat_lower +
                      0.6*solar_values_read.yhat +
                      0.3*solar_values_read.yhat_upper,3)
# just in case any forecasted value is negative, set it to 0
solar_values2[solar_values2 < 0] = 0
# change constraint RHS
mm.Params.ScenarioNumber = 3
mm.ScenNName = 'High Solar'
for t in time_periods:
    solar_avail[t].ScenNRhs = solar_values2[t]

In [60]:
mm.optimize()

#print obj value from each scenario
for s in range(mm.NumScenarios):
    # Set the scenario number to query the information for this scenario
    mm.Params.ScenarioNumber = s
    print('\nTotal cost for '+mm.ScenNName+' is '+'$'+str(round(mm.ScenNObjVal,2)))

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1440 rows, 1800 columns and 4498 nonzeros
Model fingerprint: 0x00b35a48
Variable types: 1440 continuous, 360 integer (360 binary)
Coefficient statistics:
  Matrix range     [9e-01, 2e+01]
  Objective range  [6e-02, 3e+00]
  Bounds range     [1e+00, 8e+01]
  RHS range        [1e-01, 9e+01]

Solving a multi-scenario model with 4 scenarios...

Found heuristic solution: objective 2722.4370000
Presolve removed 103 rows and 89 columns
Presolve time: 0.01s
Presolved: 1872 rows, 1889 columns, 5668 nonzeros
Presolved model has 4 scenario(s)
Found heuristic solution: objective 2713.0375100
Variable types: 1549 continuous, 340 integer (340 binary)
Found heuristic solution: objective 868.9568220

Root relaxation: objective

In [ ]:
# Limit how many solutions to collect
mm.setParam(GRB.Param.PoolSolutions, 250)
# Limit the search space by setting a gap for the worst possible solution that will be accepted
mm.setParam(GRB.Param.PoolGap, 0.05)
# do a systematic search for the k-best solutions
mm.setParam(GRB.Param.PoolSearchMode, 2)

mm.optimize()